In [1]:
import polars as pl
import polars_ds as pds

In [2]:
df = pl.DataFrame(
    {
        "id": [1, 2, 3],
        "val1": [0.1, 0.2, 5.0],
        "val2": [0.1, 0.3, 10.0],
        "val3": [0.1, 0.4, 11.0],
    }
)
df.head()

id,val1,val2,val3
i64,f64,f64,f64
1,0.1,0.1,0.1
2,0.2,0.3,0.4
3,5.0,10.0,11.0
